In [1]:
from array import array
import subprocess
import math
import ROOT
import os
import numpy as np

Welcome to JupyROOT 6.29/01


In [2]:
buildcuda = '/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/'

In [3]:
buildoneapi = '/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/'

In [4]:
buildopensycl = '/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/'

In [5]:
!ls /home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx

CMakeFiles	 InvariantMassesSYCL  SInvariantMassesSYCL
InvariantMasses  SInvariantMasses     cmake_install.cmake


In [6]:
!ls /home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx

CMakeFiles	 InvariantMassesSYCL  SInvariantMassesSYCL
InvariantMasses  SInvariantMasses     cmake_install.cmake


In [7]:
!echo $LD_LIBRARY_PATH

/opt/intel/oneapi/tbb/2021.10.0/env/../lib/intel64/gcc4.8:/opt/intel/oneapi/debugger/2023.2.0/gdb/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/libipt/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/dep/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/x64:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/oclfpga/host/linux64/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/compiler/lib/intel64_lin:/home/mdessole/Projects/AdaptiveCpp/AdaptiveCpp_install/lib:/usr/local/cuda-11.8/lib64:/home/mdessole/Projects/ROOT/root/lib:/opt/rocm-5.7.0/lib:/opt/rocm-5.7.0/opencl/lib


In [8]:
sizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [9]:
Ssizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000,200000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [10]:
def result2list(exe, N, nruns):
    print(exe, str(int(N)), str(int(nruns)))
    result = subprocess.run([exe, str(int(N)), str(int(nruns)) ], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(result)
    if len(result)<=0:
        return []
    result = result.replace("\r","").replace("\n","")
    time = np.zeros(nruns)
    for r in range(nruns):
        i = result.find("time")
        j = result.find("(s)")
        if (i<0) or (j<0):
            break
        time[r] = float(result[i+5:j])
        result = result[j+3:]
    #endif
    return time

def run_benchmark(builddir, sizes, nruns = 1, testname = "InvariantMasses", btype = 'cpu'):
    timeopt = array( 'd' )
    timestd = array( 'd' )
    
    if btype == 'cpu':
        exe = os.path.join(builddir,'testx',testname)
    elif btype == 'cuda':
        exe = os.path.join(builddir,'testx',testname+'CUDA')
    elif btype == 'sycl':
        exe = os.path.join(builddir,'testx',testname+'SYCL')
    #endif
    
    if (not os.path.exists(exe)):
        print("Executable", exe, "not found!")
        return timeopt, timestd
    
    for i, N in enumerate(sizes):
        time = result2list(exe,N,nruns)
        if (len(time)<=0):
            return timeopt, timestd
        print(time, '\n')
        timeopt.append(np.amin(time))
        timestd.append(np.std(time)) 
    #endfor
    return timeopt, timestd

In [11]:
timecpu, stdcpu = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 1000 3
RVec cpu time 6e-05 (s)
RVec cpu time 6e-05 (s)
RVec cpu time 5.9e-05 (s)

[6.0e-05 6.0e-05 5.9e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 10000 3
RVec cpu time 0.000592 (s)
RVec cpu time 0.000599 (s)
RVec cpu time 0.000594 (s)

[0.000592 0.000599 0.000594] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 100000 3
RVec cpu time 0.00592 (s)
RVec cpu time 0.005959 (s)
RVec cpu time 0.005919 (s)

[0.00592  0.005959 0.005919] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 1000000 3
RVec cpu time 0.061864 (s)
RVec cpu time 0.05917 (s)
RVec cpu time 0.059118 (s)

[0.061864 0.05917  0.059118] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 5000000 3
RVec cpu time 0.313604 (s)
RVec cpu time 0.313566 (s)
RVec cpu time 0.313839 (s)

[0.313604 0.313566 0.313839] 

/home/md

In [12]:
timecpu

array('d', [5.9e-05, 0.000592, 0.005919, 0.059118, 0.313566, 0.629, 3.13529, 6.27559])

In [13]:
Stimecpu, Sstdcpu = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 1000 3
SINGLE_PRECISION defined 
RVec cpu time 4.4e-05 (s)
RVec cpu time 4.5e-05 (s)
RVec cpu time 4.4e-05 (s)

[4.4e-05 4.5e-05 4.4e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 10000 3
SINGLE_PRECISION defined 
RVec cpu time 0.000437 (s)
RVec cpu time 0.000436 (s)
RVec cpu time 0.000437 (s)

[0.000437 0.000436 0.000437] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 100000 3
SINGLE_PRECISION defined 
RVec cpu time 0.004379 (s)
RVec cpu time 0.004372 (s)
RVec cpu time 0.004391 (s)

[0.004379 0.004372 0.004391] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 1000000 3
SINGLE_PRECISION defined 
RVec cpu time 0.045048 (s)
RVec cpu time 0.043533 (s)
RVec cpu time 0.043456 (s)

[0.045048 0.043533 0.043456] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 5000000 3
SINGLE

In [14]:
timeoneapi, stdoneapi = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000672 (s)
sycl time 0.000322 (s)
sycl time 0.00023 (s)

[0.000672 0.000322 0.00023 ] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000997 (s)
sycl time 0.000509 (s)
sycl time 0.000337 (s)

[0.000997 0.000509 0.000337] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.005428 (s)
sycl time 0.004659 (s)
sycl time 0.003104 (s)

[0.005428 0.004659 0.003104] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 1000000 3

In [15]:
timeoneapi

array('d', [0.00023, 0.000337, 0.003104, 0.036729, 0.215578, 0.426324, 2.12501, 4.24707])

In [16]:
Stimeoneapi, Sstdoneapi = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000463 (s)
sycl time 0.000311 (s)
sycl time 0.000216 (s)

[0.000463 0.000311 0.000216] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000552 (s)
sycl time 0.000361 (s)
sycl time 0.000259 (s)

[0.000552 0.000361 0.000259] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.00128 (s)
sycl time 0.001019 (s)
sycl time 0.000925 (s)

[0.00128  0.001019 0.000925] 

/home/mdess

In [17]:
timeopensycl, stdopensycl = run_benchmark(buildopensycl, sizes,  nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001255 (s)
sycl time 0.000339 (s)
sycl time 0.000203 (s)

[0.001255 0.000339 0.000203] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001501 (s)
sycl time 0.000454 (s)
sycl time 0.000342 (s)

[0.001501 0.000454 0.000342] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/InvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.002958 (s)
sycl time 0.002905 (s)
sycl time 0.00326 (s)

[0.002958 0.002905 0.00326 ] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/InvariantMassesSYCL 1

In [18]:
timeopensycl

array('d', [0.000203, 0.000342, 0.002905, 0.017273, 0.079707, 0.146101, 0.713197, 1.42353])

In [19]:
Stimeopensycl, Sstdopensycl = run_benchmark(buildopensycl, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.086978 (s)
sycl time 0.000446 (s)
sycl time 0.000328 (s)

[0.086978 0.000446 0.000328] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001176 (s)
sycl time 0.000371 (s)
sycl time 0.000216 (s)

[0.001176 0.000371 0.000216] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/SInvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001908 (s)
sycl time 0.001967 (s)
sycl time 0.001335 (s)

[0.001908 0.001967 0.001335] 

/hom

In [20]:
timecuda, stdcuda = run_benchmark(buildcuda, sizes, nruns = 3, btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000282 (s)
cuda time 4.4e-05 (s)
cuda time 4.2e-05 (s)

[2.82e-04 4.40e-05 4.20e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000403 (s)
cuda time 0.000176 (s)
cuda time 0.000171 (s)

[0.000403 0.000176 0.000171] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.001596 (s)
cuda time 0.001385 (s)
cuda time 0.001248 (s)

[0.001596 0.001385 0.001248] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.011798 (s)
cuda time 0.01

In [21]:
timecuda

array('d', [4.2e-05, 0.000171, 0.001248, 0.011125, 0.055014, 0.107327, 0.531118, 1.06906])

In [22]:
timecuda

array('d', [4.2e-05, 0.000171, 0.001248, 0.011125, 0.055014, 0.107327, 0.531118, 1.06906])

In [23]:
Stimecuda, Sstdcuda = run_benchmark(buildcuda, Ssizes, nruns = 3, testname = "SInvariantMasses", btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000219 (s)
cuda time 2.9e-05 (s)
cuda time 2.9e-05 (s)

[2.19e-04 2.90e-05 2.90e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000287 (s)
cuda time 9.1e-05 (s)
cuda time 9e-05 (s)

[2.87e-04 9.10e-05 9.00e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000803 (s)
cuda time 0.000611 (s)
cuda time 0.000579 (s)

[0.000803 0.000611 0.000579] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH

In [24]:
n = len(sizes) 
speedupopensycl = array('d', [timecpu[i]/timeopensycl[i] for i in range(n)])
speeduponeapi   = array('d', [timecpu[i]/timeoneapi[i] for i in range(n)])
speedupcuda     = array('d', [timecpu[i]/timecuda[i] for i in range(n)])

In [25]:
n = len(Ssizes)
Sspeedupopensycl = array('d', [Stimecpu[i]/Stimeopensycl[i] for i in range(n)])
Sspeeduponeapi   = array('d', [Stimecpu[i]/Stimeoneapi[i] for i in range(n)])
Sspeedupcuda     = array('d', [Stimecpu[i]/Stimecuda[i] for i in range(n)])

In [26]:
print(Sspeedupopensycl)
print(Sspeeduponeapi)
print(Sspeedupcuda)

array('d', [0.13414634146341464, 2.0185185185185186, 3.2749063670411984, 6.111095485866967, 7.378386336866901, 7.695229876305058, 8.363362369978315, 8.409360720055387, 8.451675209865536])
array('d', [0.2037037037037037, 1.6833976833976834, 4.726486486486486, 6.587236622707291, 7.483672927590097, 7.914462234597981, 8.150923660539975, 8.591703921679587, 8.64744893111639])
array('d', [1.5172413793103448, 4.844444444444444, 7.550949913644215, 9.622674933569531, 10.417267184035476, 10.59243472387208, 10.84788588700258, 10.684331889120077, 10.863369976486315])


In [27]:
val = list(sizes)
sizes_gb = [v*8*4/1e9 for v in val] #(nb of doubles)*(bytes for doubles)*(4=dim of LVector)/(bytes in GB)
sizes_gb = array('d', sizes_gb)
sizes_gb

array('d', [3.2e-05, 0.00032, 0.0032, 0.032, 0.16, 0.32, 1.6, 3.2])

In [28]:
val = list(Ssizes)
Ssizes_gb = [v*4*4/1e9 for v in val] #(nb of floats)*(bytes for floats)*(4=dim of LVector)/(bytes in GB)
Ssizes_gb = array('d', Ssizes_gb)
Ssizes_gb

array('d', [1.6e-05, 0.00016, 0.0016, 0.016, 0.08, 0.16, 0.8, 1.6, 3.2])

In [29]:
p = ROOT.TGraph(len(sizes), sizes, speedupopensycl)
g = ROOT.TGraph(len(sizes), sizes, speeduponeapi)
q = ROOT.TGraph(len(sizes), sizes, speedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec")
q.GetXaxis().SetTitle( 'Input size (# doubles)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("speedup.png")

Info in <TCanvas::Print>: png file speedup.png has been created


In [30]:
len(Ssizes), Ssizes, len(Sspeedupopensycl)

(9,
 array('d', [1000.0, 10000.0, 100000.0, 1000000.0, 5000000.0, 10000000.0, 50000000.0, 100000000.0, 200000000.0]),
 9)

In [31]:
p = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupopensycl)
g = ROOT.TGraph(len(Ssizes), Ssizes, Sspeeduponeapi)
q = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec - Single Precision")
q.GetXaxis().SetTitle( 'Input size (# floats)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sspeedup.png")

Info in <TCanvas::Print>: png file Sspeedup.png has been created


In [32]:
p = ROOT.TGraphErrors(len(sizes), sizes, timeopensycl, stdopensycl)
g = ROOT.TGraphErrors(len(sizes), sizes, timeoneapi, stdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, timecuda, stdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, timecpu, stdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times")
r.GetXaxis().SetTitle( 'Input size (# doubles)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("exectimes.png")

Info in <TCanvas::Print>: png file exectimes.png has been created


In [33]:
p = ROOT.TGraphErrors(len(sizes), sizes, Stimeopensycl, Sstdopensycl)
g = ROOT.TGraphErrors(len(sizes), sizes, Stimeoneapi, Sstdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, Stimecuda, Sstdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, Stimecpu, Sstdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times - Single Precision")
r.GetXaxis().SetTitle( 'Input size (# floats)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sexectimes.png")

Info in <TCanvas::Print>: png file Sexectimes.png has been created
